In [3]:
print("Hello World")

Hello World


In [4]:
%pwd

'c:\\Users\\amman\\Documents\\Generative AI\\End-to-End-Generative-AI-Medical-Chatbot\\research'

In [5]:
import os 
os.chdir("../")
%pwd

'c:\\Users\\amman\\Documents\\Generative AI\\End-to-End-Generative-AI-Medical-Chatbot'

In [6]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF file data as documents
def load_pdf_file(pdf_filepath):
    loader = DirectoryLoader(path=pdf_filepath, glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents 


In [7]:
documents = load_pdf_file("Data/")

In [8]:
# Split the documents into chunks
def create_doc_chunks(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    doc_chunks = splitter.split_documents(docs)
    return doc_chunks 

In [9]:
doc_chunks = create_doc_chunks(documents)

In [10]:
len(doc_chunks)

5860

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
def download_huggingface_embedding_model():
    embed_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embed_model

In [8]:
embed_model = download_huggingface_embedding_model()

C:\Users\amman\AppData\Local\Temp\ipykernel_12928\3018213434.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [9]:
query_result = embed_model.embed_query("Hello world")
print("Length of embedding vector: ", len(query_result))
print('\n')
print(query_result)

Length of embedding vector:  384


[-0.03447723761200905, 0.031023213639855385, 0.006734990980476141, 0.02610895223915577, -0.03936200216412544, -0.16030248999595642, 0.06692393124103546, -0.006441502831876278, -0.04745049029588699, 0.014758865348994732, 0.07087529450654984, 0.05552753433585167, 0.019193345680832863, -0.026251327246427536, -0.010109513066709042, -0.026940496638417244, 0.022307435050606728, -0.022226642817258835, -0.1496925801038742, -0.01749304123222828, 0.007676258217543364, 0.05435232073068619, 0.0032544711139053106, 0.031725890934467316, -0.0846213549375534, -0.029405983164906502, 0.05159558728337288, 0.04812406003475189, -0.0033148040529340506, -0.05827920883893967, 0.04196924716234207, 0.022210638970136642, 0.1281888484954834, -0.022338991984725, -0.011656233109533787, 0.06292837113142014, -0.03287629410624504, -0.09122603386640549, -0.031175388023257256, 0.052699580788612366, 0.0470348484814167, -0.08420310914516449, -0.030056176707148552, -0.020744847133755684, 

In [1]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from dotenv import load_dotenv
import os 

load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)



c:\Users\amman\anaconda3\envs\medbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-ci9vpsy.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [14]:
# Embed each chunk and upsert the embeddings into the Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=doc_chunks,
    index_name=index_name,
    embedding=embed_model
)

In [10]:
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
index_name = "medicalbot"
# Load index to perform retrieval
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embed_model
)

In [11]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [12]:
retrieved_docs = retriever.invoke("What is Acne?")

In [13]:
retrieved_docs

[Document(id='cdfb7e39-bcbd-406a-80d1-8f51a7fff4a6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='77f85ce1-1590-44cd-a9e8-f4405c77b096', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [14]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [16]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. Only answer based on the provided context, not from your own knowledge. If you don't know the answer just say that you don't know. "
    "Use three sentences maximum and keep the answers concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [17]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [18]:
response = rag_chain.invoke({"input": "What is Acupressure?"})
print(response["answer"])



Acupressure is a form of touch therapy that uses the same points on the body as acupuncture, but instead of needles, finger pressure is used to stimulate these points. It is a non-invasive and cost-effective way to relieve symptoms and pain, increase energy and well-being, and stimulate the immune system. It can be performed by a layperson and is used to treat a variety of health conditions.
